# Поиск ассоциативных правил

In [1]:
import warnings
warnings.filterwarnings("ignore")

## Библиотеки

In [2]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
from copy import deepcopy
from mlxtend.plotting import plot_decision_regions
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.spatial.distance import cdist
import numpy as np
from sklearn import tree
import itertools
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.datasets import make_classification, load_wine, load_boston
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from torchvision import datasets
import pyfpgrowth
from torchvision import transforms

## Ассоциативные правила

Рассмотрим выборку:
$$
\mathbf{X} \in \left\{0, 1\right\}^{l \times n}
$$

Рассмотрим множество всех конъюнктивных предикатов над множеством признаков:
$$
\Phi = \left\{\varphi| \varphi\bigr(\mathbf{x}_i\bigr) = \wedge_{j \in \mathcal{J}}x_i^j \quad \mathcal{J} \subset \{1 \cdots n\}\right\} \qquad |\Phi| = 2^{n}
$$

Частота встречаемости набора:
$$
\nu\bigr(\varphi\bigr) = \frac{1}{l}\sum_{i=1}^{l}\varphi\bigr(\mathbf{x}_i\bigr)
$$

Минимальная поддержка $\delta$:
$$
    \nu\bigr(\varphi\bigr) \geq \delta
$$

Наборы $\varphi, \psi \in \Phi: \psi\cap\varphi=\emptyset$ совместно часто встречаемы, если:
$$
\nu\bigr(\varphi \cup \psi \bigr) \geq \delta
$$

Условная совстречаемость наборов на уровне значимости $\varkappa$:
$$
\nu\bigr(\psi | \varphi \bigr) = \frac{\nu\bigr(\varphi \cup \psi \bigr)}{\nu\bigr(\varphi\bigr)} \geq \varkappa
$$

## Простой пример

###### синтетический датасет

In [3]:
np.random.seed(0)
l = 200
X = np.vstack([np.random.choice([0, 1], size=l, p=[0.5,0.5]), 
               np.random.choice([0, 1], size=l, p=[0.5,0.5])]).T

X = np.hstack([X, 
               X*np.random.choice([0, 1], size=[l, 2], p=[0.2,0.8])])

###### генерим все возможные наборы

In [4]:
Phi = list(itertools.product([0,1], repeat=4))
kappa = delta = 0.35

result = []
deltas = []
for phi in Phi:
    res = np.sum(
        ((X*phi).sum(axis=-1) == np.sum(phi, axis=-1))\
        *np.array(np.sum(phi, axis=-1), dtype=np.bool))/l
    
    result.append(int(res >= delta))
    deltas.append(res)

###### получаем список частых наборов

In [5]:
list(zip(Phi, result, deltas))

[((0, 0, 0, 0), 0, 0.0),
 ((0, 0, 0, 1), 1, 0.37),
 ((0, 0, 1, 0), 1, 0.445),
 ((0, 0, 1, 1), 0, 0.14),
 ((0, 1, 0, 0), 1, 0.475),
 ((0, 1, 0, 1), 1, 0.37),
 ((0, 1, 1, 0), 0, 0.175),
 ((0, 1, 1, 1), 0, 0.14),
 ((1, 0, 0, 0), 1, 0.53),
 ((1, 0, 0, 1), 0, 0.175),
 ((1, 0, 1, 0), 1, 0.445),
 ((1, 0, 1, 1), 0, 0.14),
 ((1, 1, 0, 0), 0, 0.225),
 ((1, 1, 0, 1), 0, 0.175),
 ((1, 1, 1, 0), 0, 0.175),
 ((1, 1, 1, 1), 0, 0.14)]

###### какие из частых наборов являются совместно частыми

In [6]:
list(zip(Phi, result, deltas))

[((0, 0, 0, 0), 0, 0.0),
 ((0, 0, 0, 1), 1, 0.37),
 ((0, 0, 1, 0), 1, 0.445),
 ((0, 0, 1, 1), 0, 0.14),
 ((0, 1, 0, 0), 1, 0.475),
 ((0, 1, 0, 1), 1, 0.37),
 ((0, 1, 1, 0), 0, 0.175),
 ((0, 1, 1, 1), 0, 0.14),
 ((1, 0, 0, 0), 1, 0.53),
 ((1, 0, 0, 1), 0, 0.175),
 ((1, 0, 1, 0), 1, 0.445),
 ((1, 0, 1, 1), 0, 0.14),
 ((1, 1, 0, 0), 0, 0.225),
 ((1, 1, 0, 1), 0, 0.175),
 ((1, 1, 1, 0), 0, 0.175),
 ((1, 1, 1, 1), 0, 0.14)]

###### какие из частых наборов являются условно частыми

In [7]:
list(zip(Phi, result, deltas))

[((0, 0, 0, 0), 0, 0.0),
 ((0, 0, 0, 1), 1, 0.37),
 ((0, 0, 1, 0), 1, 0.445),
 ((0, 0, 1, 1), 0, 0.14),
 ((0, 1, 0, 0), 1, 0.475),
 ((0, 1, 0, 1), 1, 0.37),
 ((0, 1, 1, 0), 0, 0.175),
 ((0, 1, 1, 1), 0, 0.14),
 ((1, 0, 0, 0), 1, 0.53),
 ((1, 0, 0, 1), 0, 0.175),
 ((1, 0, 1, 0), 1, 0.445),
 ((1, 0, 1, 1), 0, 0.14),
 ((1, 1, 0, 0), 0, 0.225),
 ((1, 1, 0, 1), 0, 0.175),
 ((1, 1, 1, 0), 0, 0.175),
 ((1, 1, 1, 1), 0, 0.14)]

## Свойство антимонотонности

В приведенном выше примере легко заметить свойство антиминотонности.

$$
    \forall \varphi, \psi \in \Phi: \varphi \subset \psi \Rightarrow \nu\bigr(\varphi\bigr) \geq \nu\bigr(\psi\bigr)
$$

Получаем следствия, которые также легко наблюдаемы в приведенном выше примере:
1. $\varphi$ частый, тогда $\forall \psi \subset \varphi$ следует, что $\psi$ частый.
2. $\psi$ не частый, тогда $\forall \psi \subset \varphi$ следует, что $\varphi$ не частый.
3. $\nu\bigr(\varphi\cup \psi\bigr) \leq \nu\bigr(\varphi\bigr)$


## Алгоритм APriory
Реализация взята с [сайта](https://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/).

In [8]:
dataSet = [np.where(x == 1)[0].tolist() for x in X]

In [9]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))

In [10]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [11]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [12]:
def apriori(dataSet, minSupport):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [13]:
L, suppData = apriori(dataSet, delta)

In [14]:
L

[[frozenset({3}), frozenset({1}), frozenset({2}), frozenset({0})],
 [frozenset({1, 3}), frozenset({0, 2})],
 []]

In [15]:
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

In [16]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [17]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [18]:
rules = generateRules(L, suppData, minConf=0.7)

frozenset({3}) --> frozenset({1}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 0.7789473684210526
frozenset({2}) --> frozenset({0}) conf: 1.0
frozenset({0}) --> frozenset({2}) conf: 0.8396226415094339


## Псевдореальные данные 

Как говорилось ранее, постановка задачи поиска асоциативных правил берет начало из задачи поиска закономерстей в чеках покупателей. Постановка задачи простая: нужно понять какие продукты покупатели покупают вместе.

Данные чеков покупок взята с одного из конкурсов [kaggle](https://www.kaggle.com/shwetabh123/basketballoptimisation).


In [19]:
dataset = pd.read_csv('data/data.csv', header = None)

dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
dataset.fillna(method = 'ffill', axis = 1, inplace = True)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs
2,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney
3,turkey,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado
4,mineral water,milk,energy bar,whole wheat rice,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea


In [21]:
transactions = []
for i in range(0, len(dataset)): 
    transactions.append(
        np.unique(
            [str(
                dataset.values[i,j]) 
             for j in range(0, len(dataset.columns))]).tolist())


In [22]:
word_to_ind = dict()
ind_to_word = dict()

for word in np.unique(list(itertools.chain(*transactions))).tolist():
    word = word.strip()
    key = word_to_ind.__len__()
    word_to_ind[word] = key
    ind_to_word[key] = word

In [23]:
transactions_ind = []

for transaction in transactions:
    indexes = []
    for word in transaction:
        word = word.strip()
        indexes.append(word_to_ind[word])
        
    transactions_ind.append(indexes)

In [24]:
delta = 1e-3
kappa = 0.47
L, suppData = apriori(transactions_ind, delta)

In [25]:
rules = generateRules(L, suppData, minConf=kappa)

frozenset({67}) --> frozenset({71}) conf: 0.4782608695652174
frozenset({103}) --> frozenset({99}) conf: 0.4827586206896552
frozenset({77}) --> frozenset({71}) conf: 0.4871794871794871
frozenset({62}) --> frozenset({71}) conf: 0.4700854700854701
frozenset({40}) --> frozenset({71}) conf: 0.47777777777777775
frozenset({16, 68}) --> frozenset({70, 71}) conf: 0.5714285714285714
frozenset({80, 60}) --> frozenset({99, 71}) conf: 0.4782608695652174
frozenset({106, 53}) --> frozenset({99, 54}) conf: 0.5263157894736842
frozenset({70, 103}) --> frozenset({99, 71}) conf: 0.47058823529411764
frozenset({88, 30}) --> frozenset({99, 71}) conf: 0.47058823529411764
frozenset({83, 71}) --> frozenset({96, 36}) conf: 0.625
frozenset({83, 36}) --> frozenset({96, 71}) conf: 0.47619047619047616
frozenset({109, 7}) --> frozenset({99, 36}) conf: 0.5333333333333332
frozenset({96, 48, 54}) --> frozenset({99, 71}) conf: 0.5416666666666666
frozenset({48, 58, 99}) --> frozenset({54, 71}) conf: 0.47368421052631576
fr

In [26]:
for rule in rules:
    phi = []
    psi = []
    for item in rule[0]:
        phi.append(ind_to_word[item])
        
    for item in rule[1]:
        psi.append(ind_to_word[item])
        
    print('{} --> {}'.format(';'.join(phi), ';'.join(psi)))

mayonnaise --> mineral water
strong cheese --> spaghetti
nonfat milk --> mineral water
light cream --> mineral water
extra dark chocolate --> mineral water
cake;meatballs --> milk;mineral water
olive oil;hot dogs --> spaghetti;mineral water
tomato sauce;green tea --> spaghetti;ground beef
milk;strong cheese --> spaghetti;mineral water
red wine;cooking oil --> spaghetti;mineral water
pasta;mineral water --> shrimp;eggs
pasta;eggs --> shrimp;mineral water
turkey;black tea --> spaghetti;eggs
shrimp;frozen vegetables;ground beef --> spaghetti;mineral water
frozen vegetables;herb & pepper;spaghetti --> ground beef;mineral water
shrimp;frozen vegetables;olive oil --> chocolate;mineral water
olive oil;chocolate;tomatoes --> spaghetti;mineral water
frozen vegetables;soup;olive oil --> milk;mineral water


## Алгоритм FP-Growth

In [27]:
patterns = pyfpgrowth.find_frequent_patterns(transactions_ind, 4)

In [28]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)

In [29]:
for key in np.random.choice(list(rules.keys()), size=15):
    phi = []
    psi = []
    for item in key:
        phi.append(ind_to_word[item])
        
    for item in rules[key][0]:
        psi.append(ind_to_word[item])
        
    print('{} --> {}'.format(';'.join(phi), ';'.join(psi)))

burgers;milk;tomatoes;whole wheat rice --> spaghetti
chocolate;shrimp;spaghetti;tomato sauce --> ground beef
burgers;eggs;herb & pepper;milk;mineral water --> ground beef
ground beef;pancakes;whole weat flour --> milk;mineral water
eggs;french fries;frozen vegetables;ground beef --> spaghetti
chocolate;eggs;fresh bread;frozen vegetables --> mineral water
cooking oil;eggs;green tea;mineral water --> spaghetti
cooking oil;mineral water;soup;spaghetti --> eggs
frozen vegetables;milk;mineral water;olive oil;tomatoes --> spaghetti
chicken;chocolate;herb & pepper;soup --> ground beef
chocolate;eggs;shrimp;turkey --> mineral water
cake;eggs;meatballs;milk --> mineral water
eggs;frozen vegetables;low fat yogurt;mineral water --> milk
carrots;pancakes;soup --> mineral water
cake;frozen vegetables;olive oil;shrimp --> mineral water


## Поиск ассоциативных правила для вещественных данных

Ранее были рассмотрены методы для поиска ассоциативных правил в случае категориальных признаков. Для обобщения всех выше методов для вещественных признаков легко вводится квантизация данных:

1. Например для вещественного признака `стоимость` можно выполнить квантизацию:
    - x < 500
    - 500 < x < 1000
    - 1000 < x < 10000
    - 10000 < x
    
2. Для вещественного признака `количество` квантизация также легко выполняется:
    - x < 2
    - 2 < x < 10
    - 10 < x
    
Введя квантизацию задача на вещественных данных сведена к задаче на категориальных данных. Заметим, что каждый вещественный признак нужно анализировать отдельно.

## Обобщенные ассоциативные правила

Ранее мы рассматривали корзину покупателей. При анализе данной корзины мы предполагали, что все данные однородны (все измеряли в товарах).

Иногда полезно анализировать группы товаров. Примерами груп может служить:
1. Безалкогольные напитки.
2. Молочные продукты.

В случае, когда требуется строить ассоциативные правила вида `Молочные продукты -> спагети`, где одна часть является группой товаров, а вторая часть является товаром из другой группы называется построение обобщенных ассоциативных правил.

Простым подходом для построение обобщенных ассоциативных правил является добавления всех групп продуктов в признаковое описание. Такое решение является плохим, так как:
1. В группах минимальная поддержка требуется больше чем в отдельных предметах.
2. Добавление всевозможных групп товаров расширяет пространство признаков.
3. Очевидно, что при таком подходе и применению простых алгоритмов получим большое количество нерелевантных правил вид: `Молоко -> Молочные продукты`.

Построение обобщенных ассоциативных правил требует дополнительного исследования. Много работ для их построения вводят специальные ограничения на вид правил ("отсеивая" нерелевантные).